In [18]:
import pandas as pd 
import numpy as np
import pyblp
from sklearn.linear_model import LinearRegression

In [19]:
product_data = pd.read_csv('../data/market_integrates_2.csv')
print(product_data)

     market_ids  firm_ids  characteristic1  characteristic2    prices  \
0             1         1        12.239192         1.107516  2.949501   
1             1         2        11.977273         1.290293  3.372321   
2             1         3        11.897650         1.238727  3.497949   
3             1         4        12.359558         1.218664  2.634126   
4             1         5        11.908647         1.170831  3.450975   
..          ...       ...              ...              ...       ...   
995         100         6        12.272833         1.328612  2.707096   
996         100         7        12.137848         1.461094  2.818468   
997         100         8        11.921876         1.136990  3.452236   
998         100         9        11.772948         1.345224  3.526833   
999         100        10        12.168702         1.496973  2.494795   

     marginal_cost    shares   profits   markups  e_quantity   capital  \
0         1.855760  0.021463  0.023475  1.589376 

In [20]:
# # Getting the dataset estimation ready with at least a supply_side instrument/ demand_side instrument 
# product_data_complete = product_data.rename(columns={'labor':'supply_instrument0',
#                                                     #   'capital':'supply_instruments1',
#                                                         # 'marginal_cost': 'supply_instrument2'
#                                                         })



In [21]:
# the endog regressor
X1_formulation = pyblp.Formulation('1  + prices+ characteristic1 + characteristic2')

# the only regressor on which we want random coeff 
X2_formulation = pyblp.Formulation('0 +  prices')

# supply side charact 
# X3_formulation = pyblp.Formulation('1 + log(labor) + log(capital)')

# formulation for the cost thing (assume cost fomrulation being labor and capital)
product_formulations = (X1_formulation,
                         X2_formulation
                           #  X3_formulation
                           )
print(product_formulations)

(1 + prices + characteristic1 + characteristic2, prices)


In [22]:
mc_integration = pyblp.Integration('monte_carlo', size=50, specification_options={'seed': 0})


In [23]:
mc_problem = pyblp.Problem(product_formulations, product_data, integration=mc_integration, add_exogenous=True)
print(mc_problem)

Initializing the problem ...
Initialized the problem after 00:00:00.

Dimensions:
 T    N     F    I     K1    K2    MD 
---  ----  ---  ----  ----  ----  ----
100  1000  10   5000   4     1     3  

Formulations:
       Column Indices:           0       1            2                3       
-----------------------------  ------  ------  ---------------  ---------------
 X1: Linear Characteristics      1     prices  characteristic1  characteristic2
X2: Nonlinear Characteristics  prices                                          
Dimensions:
 T    N     F    I     K1    K2    MD 
---  ----  ---  ----  ----  ----  ----
100  1000  10   5000   4     1     3  

Formulations:
       Column Indices:           0       1            2                3       
-----------------------------  ------  ------  ---------------  ---------------
 X1: Linear Characteristics      1     prices  characteristic1  characteristic2
X2: Nonlinear Characteristics  prices                                          


In [24]:
bfgs = pyblp.Optimization('bfgs', {'gtol': 1e-4})


In [25]:
results1 = mc_problem.solve(sigma=np.ones((1, 1)), optimization=bfgs)


Solving the problem ...

Nonlinear Coefficient Initial Values:
Sigma:     prices    
------  -------------
prices  +1.000000E+00
Starting optimization ...

GMM   Optimization   Objective   Fixed Point  Contraction  Clipped    Objective      Objective      Gradient                  
Step   Iterations   Evaluations  Iterations   Evaluations  Shares       Value       Improvement       Norm           Theta    
----  ------------  -----------  -----------  -----------  -------  -------------  -------------  -------------  -------------
 1         0             1           608         1924         0     +6.644902E-19                 +1.458146E-08  +1.000000E+00

Optimization completed after 00:00:00.
Computing the Hessian and and updating the weighting matrix ...
Computed results after 00:00:01.

Problem Results Summary:
GMM     Objective      Gradient                    Clipped  Weighting Matrix
Step      Value          Norm          Hessian     Shares   Condition Number
----  -------------